### Imports

In [ ]:
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import matplotlib
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder


%matplotlib inline

matplotlib.style.use('ggplot')

### Running the Modules Implementation Notebook

In [ ]:
# note that if you change smth there, you have to run this again
%run NN_Implementation_Modules.ipynb

### Implementation of Stochastic Gradient Descent

Feel free to add prints for understanding in "debug-mode" :)

In [ ]:
def sgd(x, dx, lr, state={}):
    """
    Implementation of sgd with storing old gradients in state. Google to understand how setdefault works.

    :param lr:     learning rate, default value is 0.1
    :param state:  empty dictionary to store gradients throught the update process
    """
    
    state.setdefault('old_grad', {})
    
    i = 0 
    for current_layer_x, current_layer_dx in zip(x,dx): 
        for current_x, current_dx in zip(current_layer_x,current_layer_dx):
            current_old_grad = state['old_grad'].setdefault(i, np.zeros_like(current_dx))
            current_old_grad = current_old_grad + current_dx
            state['old_grad'][i] = current_old_grad
            np.add(current_x, lr*(-current_old_grad), out=current_x)
            i += 1     

# Toy Example

Simple example for debugging implementation.

### Data Generation

In [ ]:
# Generate some data
N = 500

X1 = np.random.randn(N,2) + np.array([2,2])
X2 = np.random.randn(N,2) + np.array([-2,-2])

Y = np.concatenate([np.ones(N),np.zeros(N)])[:,None]
Y = np.hstack([Y, 1-Y]) # one-hot encoding Y

X = np.vstack([X1,X2])
plt.scatter(X[:,0],X[:,1], c = Y[:,0], edgecolors= 'none')

### Network Architecture

Try architecture that will perform `logistic regression` first, then try something like the following. 

In [ ]:
criterion = MSECriterion()

net = Sequential()
net.add(Linear(2, 10))
net.add(Relu())
net.add(Linear(10, 2))
net.add(SoftMax())

print(net)

### Some Hyperparameters
At first, try using the whole dataset as batch and make sure that loss reduces at each step. Then reduce batch size and examine results.

Play with number of epochs and learning rate.

In [ ]:
n_epoch = 10
batch_size = 1000
lr = 1e-1

### Batch Generator

Google to understand how python's `generator` objet differs from iterator.


In [ ]:
def get_batches(X, Y, batch_size):
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx]

### Train

In [ ]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in get_batches(X, Y, batch_size):
        net.zeroGradParameters()
        
        # Forward Pass
        predictions = net.forward(x_batch)
        loss = criterion.forward(predictions, y_batch)
    
        # Backward Pass
        bp = criterion.backward(predictions, y_batch)
        net.backward(x_batch, bp)
        
        # Update weights
        sgd_momentum(net.getParameters(), 
                     net.getGradParameters(),
                     lr)
           
        loss_history.append(loss)

    # Visualize
    display.clear_output(wait=True)
    plt.figure(figsize=(8, 6))
        
    plt.title("Training loss")
    plt.xlabel("#iteration")
    plt.ylabel("loss")
    plt.plot(loss_history, 'b')
    plt.show()
    
    print('Current loss: %f' % loss)    

# Digit classification 

### Fetching the Dataset

In [ ]:
if os.path.exists('mnist.npz'):
    with np.load('mnist.npz', 'r', allow_pickle=True) as data:
        train_images = data['X']
        train_labels = data['y']
else:
    mnist = fetch_openml("mnist_784")
    train_images, train_labels = mnist.data / 255.0, mnist.target
    np.savez('mnist.npz', X=X, y=y)

### One-hot Encoding the Labels
For one-hot encoding you can use sklearn's `OneHotEncoder` class.

In [ ]:
enc = OneHotEncoder()
one_hot_y = enc.fit_transform(train_labels.reshape(-1, 1)).todense()

### Train Test Splitting

In [ ]:
X_train = train_images[:10000]/255
y_train = one_hot_y[:10000]
y_train = np.array(y_train)/255
X_val = train_images[10000:15000]/255
y_val = one_hot_y[10000:15000]
y_val = np.array(y_val)/255

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_val', X_val.shape)
print('y_val', y_val.shape)


### Training a NN

You can use the codes above.

In [ ]:
# TODO: Define Hyperparameters here

In [ ]:
# TODO: Define NN architecture 

In [ ]:
# TODO: Training Proccess

### Evaluate on Test Dataset

In [ ]:
net.evaluate()
pred = net.forward(X_val)
pred = np.argmax(pred, axis=1)
accuracy_score(pred, np.argmax(y_val, axis=1))